# Защита персональных данных клиентов

Необходимо защитить данные клиентов страховой компании «Хоть потоп». Разработаем такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score

In [2]:
df = pd.read_csv("/datasets/insurance.csv")
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [3]:
df.duplicated().sum()

153

In [4]:
df.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

В наших данных нет пропусков, но есть 153 дубликата. Так как в проекте не требуется подбирать наилучшую модель, то и дубликаты можно оставить.

Разделим наши данные на признаки и целевой признак.

In [5]:
features = df.drop("Страховые выплаты", axis=1)
target = df["Страховые выплаты"]

features.shape, target.shape

((5000, 4), (5000,))

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Если в задаче линеной регрессии $a = Xw$ умножить $X (M, N)$ на обратимую матрицу $P$, размерности $(N, N)$, то при вычислении весов $w = (X^T X)^{-1} X^T y$ предсказания $a$ не дожны изменится.


**Обоснование:** 
Для доказательства подставим  $XP$ вместо $X$

$$
a_{new} = XPw_{new}
$$$$
w_{new} = ((XP)^T XP)^{-1}(XP)^T y
$$
подставим $w_{new}$ в $a_{new}$ и преобразуем:

$$
a_{new} = X P ((XP)^T XP)^{-1}(XP)^T y = \\X P (P^TX^T XP)^{-1}P^T X^T y = \\X P (X^TXP)^{-1} (P^T)^{-1} P^T X^T y = \\X P P^{-1}(X^TX)^{-1} (P^T)^{-1} P^T X^T y = \\X E (X^TX)^{-1} E  X^T y = \\X (X^TX)^{-1} X^T y =  X w
$$
Пояснения:

Раскрываем транспонирования по свойству: $(AB)^T = B^T A^T$  
Считаем $X^TX$ - квадратная обратимая марица, $P$ - обратимая по опеределнию, то раскрываем скобки по свойству $(AB)^{-1} = B^{-1} A^{-1}$ 2 раза  
Используя ассоциативность умножения матриц и определение обратной матрицы получим ответ  
Таким образом получаем что:  

$$  
a_{new} = a  
$$  
Во всех этих рассуждениях одно допущение, что $X^TX$ - обратима, но в общем случае она может оказаться не обратимой  

## Алгоритм преобразования

**Алгоритм**

Для точго, чтобы зашифровать персональные данные клиентов воспользуемся таким алгоритмом:  
1. Создадим рандомную матрицу $P$  4х4
2. Проверим её на обратимость  
3. Создадим новую матрицу данных $M$  
4. Посчитаем метрики качества $R2 score$ и $MSE$ на исходных данных и на преобразованных  
5. Обрадуемся результату 


**Обоснование**

Если мы произведём умножение признаков тренировочной и тестовой выборок на одну и ту же обратимую матрицу, то результатом предсказания будет произведение нашей изменённой таблицы с признаками на вектор, который был получен из той же таблицы с изменёнными данными увеличенную на ту же величину, что и признаки тренировочных данных. 

## Проверка алгоритма

Проверим, обратима ли матрица и напишем функцию для вычисления метрики Р2 для модели линейной регрессии. Затем посчитаем эту метрику для обычной матрицы признаков и для обратимой.

In [6]:
INV = np.random.rand(4,4)
np.linalg.inv(INV)

array([[ 0.22223876,  1.73455824, -0.94010328, -0.21974737],
       [-1.3021401 , -0.40035133,  0.57916761,  1.69543025],
       [ 3.45621695, -1.79721217, -0.70918087, -1.08949482],
       [-0.98083969,  0.9014485 ,  1.56908806, -0.88551723]])

In [7]:
def predict_and_r2_score(features, target):
    lr = LinearRegression()
    lr.fit(features, target)
    r2 = lr.score(features, target)
    return r2

In [8]:
predict_and_r2_score(features, target)

0.42494550286668

In [9]:
features_inv = features @ INV
predict_and_r2_score(features_inv, target)

0.42494550286667165

Мы увидели, что качество линейной регрессии не меняется при умножении матрицы признаков NxK справа на квадратную обратимую матрицу KxK.

Наконец, определим тестовую и тернировочную выборки для дальнейшего обучения. 

In [10]:
train_features, test_features, train_target, test_target = train_test_split(features,target, test_size=0.25, random_state=254)

Создадим матрицу $P$ и проверим ее обратимость 

In [11]:
P = np.random.normal(3, size = (4,4))
P

array([[4.19739309, 3.12481934, 5.0173566 , 3.58918589],
       [2.1913364 , 0.48685376, 4.51322433, 2.22463007],
       [3.36736614, 2.98985369, 2.77113557, 2.90156699],
       [3.10112218, 3.2378757 , 2.41649999, 3.2181574 ]])

In [12]:
np.random.seed(seed=12345)
np.random.normal(3, size = (4,4))

array([[2.79529234, 3.47894334, 2.48056128, 2.4442697 ],
       [4.96578057, 4.39340583, 3.09290788, 3.28174615],
       [3.76902257, 4.24643474, 4.00718936, 1.70377889],
       [3.27499163, 3.22891288, 4.35291684, 3.88642934]])

In [13]:
np.linalg.inv(P)

array([[-1.93180616,  0.70511356,  4.18945623, -2.11021168],
       [ 2.490514  , -1.38756296, -3.10468252,  0.98079133],
       [ 1.56674407, -0.51490738, -1.98294013,  0.39643069],
       [-1.82068313,  1.1032359 ,  0.57558877,  1.05972724]])

Теперь выполним умножение признаков на новую матрицу и обозначим ее $M$. Затем проделаем тоже самое с тестовой выборкой. 

In [14]:
M = train_features.dot(P)
M.columns = train_features.columns
M.head()

,Пол,Возраст,Зарплата,Члены семьи
883,132723.345151,117813.697198,109279.935804,114371.674808
4736,167809.507833,148924.932301,138231.133406,144613.850335
4455,139479.990798,123798.273116,114867.339194,120197.055090
3993,167417.987808,148613.775588,137841.189240,144268.077885
651,137136.520538,121716.062868,112943.820241,118180.061929


In [15]:
test_M = test_features.dot(P)
test_M.columns = test_features.columns
test_M.head()

,Пол,Возраст,Зарплата,Члены семьи
4948,128048.291329,113638.816511,105477.574947,110348.664289
4959,116280.763827,103183.838526,95807.911929,100212.592889
1728,127352.180231,113031.075102,104884.321405,109745.971137
4169,147889.640805,131270.959927,121777.125230,127442.074045
1984,150279.295385,133376.175506,123775.911824,129506.302814


Целевые признаки оставляем без изменений. А в нулевой столбец измененной таблицы внесем значение 1 

In [16]:
X = np.concatenate((np.ones((M.shape[0], 1)), M), axis=1)

In [17]:
pd.DataFrame(X).head()

,0,1,2,3,4
0,1.0,132723.345151,117813.697198,109279.935804,114371.674808
1,1.0,167809.507833,148924.932301,138231.133406,144613.850335
2,1.0,139479.990798,123798.273116,114867.339194,120197.055090
3,1.0,167417.987808,148613.775588,137841.189240,144268.077885
4,1.0,137136.520538,121716.062868,112943.820241,118180.061929


Присвоим переменной $у$, целевые признаки и вычислим по формуле ( $w = (X^T X)^{-1} X^T y$ ) вектор $w$

Затем вычислим предсказание модели методом умножения тестовой выбрки $test_M$ и увеличим на величину сдвига значение $w0$  
Переменную предсказанных значений обозначим $a$

In [18]:
y = train_target
w = (np.linalg.inv(X.T.dot(X)).dot(X.T)).dot(y)

In [19]:
a = test_M.dot(w[1:]) + w[0]

Посчитаем значения R2 и MSE на исходных данных и на измененных.

In [20]:
model_l = LinearRegression()
model_l.fit(train_features,train_target)
predictions= model_l.predict(test_features)

R2_score_l = r2_score(test_target, predictions)
MSE_l = mean_squared_error(predictions,test_target)

In [21]:
R2_score_m = r2_score(test_target, a)
MSE_m = mean_squared_error(a, test_target)
print("R2_score:",R2_score_m)
print("MSE:", MSE_m)

R2_score: 0.41668736296246567
MSE: 0.11248918214706362


In [22]:
comparison_tabl = pd.DataFrame(index=['R2_score','MSE'], 
                               columns=['Изменённая матрица','Неизменённая матрица'])
comparison_tabl['Изменённая матрица'] = R2_score_m, MSE_m
comparison_tabl['Неизменённая матрица'] = R2_score_l, MSE_l

comparison_tabl.T

,R2_score,MSE
Изменённая матрица,0.416687,0.112489
Неизменённая матрица,0.416687,0.112489


### Вывод 

В проекте требовалось защитить данные клиентов страховой коммпании. Преобразование необходимо было произвести таким образом, чтобы по результату работы нашего алгоритма, качество моделей машинного обучения не ухудшилось.

В результате нашей работы был разработан алгоритм, где данные были преобразованы методом умножения признаков исходных данных на обратимую матрицу $P$. Результатом такого преобразования явилась новая (изменённая) матрица признаков, которая справилась с основными задачами по шифровке данных и при сравнении качества предсказаний с моделью из Scikit-learn, сохранила значения R2 и MSE неизменными.